In [16]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from functools import cache
from update_vars import GCS_FILE_PATH, analysis_date
@cache
def gcs_geopandas():
    return GCSGeoPandas()

In [2]:
from shared_utils import webmap_utils, catalog_utils
from calitp_data_analysis.geography_utils import WGS84, CA_NAD83Albers_m

# Check GTFS station entrance coverage

* Mainly BART and LA Metro

In [3]:
df = query_sql('select distinct gtfs_dataset_name from cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds where date = "2025-08-18"')

In [4]:
df[df.gtfs_dataset_name.str.contains('Muni')]

,gtfs_dataset_name
131,Bay Area 511 Muni Schedule


In [37]:
query = '''
SELECT t1.gtfs_dataset_name, t1.gtfs_dataset_key as schedule_gtfs_dataset_key_primary, t1.date, t2.feed_key, t2.stop_id,
t2.parent_station, t2.stop_code, t2.stop_name, t2.stop_desc, t2.location_type, t2.pt_geom
FROM cal-itp-data-infra.mart_gtfs.fct_daily_schedule_feeds AS t1
INNER JOIN cal-itp-data-infra.mart_gtfs.dim_stops AS t2
ON t1.feed_key = t2.feed_key
WHERE t1.gtfs_dataset_name IN ("LA Metro Rail Schedule", "Bay Area 511 BART Schedule", "Metrolink Schedule",
"Bay Area 511 Sonoma-Marin Area Rail Transit Schedule", "Sacramento Schedule", "San Diego Schedule", "Bay Area 511 Muni Schedule")
AND t1.date = '2026-01-14' AND t2.location_type = 2
'''

In [40]:
df2 = query_sql(query)

In [41]:
gdf = gpd.GeoDataFrame(df2, geometry=gpd.GeoSeries.from_wkt(df2.pt_geom), crs=WGS84)

In [42]:
gdf

,gtfs_dataset_name,schedule_gtfs_dataset_key_primary,date,feed_key,stop_id,parent_station,stop_code,stop_name,stop_desc,location_type,pt_geom,geometry
0,LA Metro Rail Schedule,7ada6c55e4a29f4535e84c504a994b14,2026-01-14,5ad2c66ffc1fa1e41f64c46b681524d5,80105A,80105S,80105A,Anaheim Street Station - Long Beach & 12th Ent...,None,2,POINT(-118.18939 33.78135),POINT (-118.18939 33.78135)
1,LA Metro Rail Schedule,7ada6c55e4a29f4535e84c504a994b14,2026-01-14,5ad2c66ffc1fa1e41f64c46b681524d5,80138A,80138S,80138A,17th Street / SMC Station - East Ramp,None,2,POINT(-118.47982 34.02363),POINT (-118.47982 34.02363)
2,LA Metro Rail Schedule,7ada6c55e4a29f4535e84c504a994b14,2026-01-14,5ad2c66ffc1fa1e41f64c46b681524d5,80414A,80414S,80414A,Highland Park Station - Ave 57 Entrance,None,2,POINT(-118.19356 34.1107),POINT (-118.19356 34.11070)
3,LA Metro Rail Schedule,7ada6c55e4a29f4535e84c504a994b14,2026-01-14,5ad2c66ffc1fa1e41f64c46b681524d5,80405A,80405S,80405A,Soto Station - Main Entrance,None,2,POINT(-118.21019 34.04364),POINT (-118.21019 34.04364)
4,LA Metro Rail Schedule,7ada6c55e4a29f4535e84c504a994b14,2026-01-14,5ad2c66ffc1fa1e41f64c46b681524d5,80702A,80702S,80702A,LAX / Metro Transit Center - Main Pedestrian E...,None,2,POINT(-118.37816 33.95002),POINT (-118.37816 33.95002)
...,...,...,...,...,...,...,...,...,...,...,...,...
356,Bay Area 511 BART Schedule,1ee91bdb321ed2f8e617449cdc1b770a,2026-01-14,4d1c677d47f3c48a4323f506927383d3,ORIN_1,903209,ORIN_1,Main Entrance / Exit,Main Entrance / Exit,2,POINT(-122.1840791 37.87811675),POINT (-122.18408 37.87812)
357,Bay Area 511 BART Schedule,1ee91bdb321ed2f8e617449cdc1b770a,2026-01-14,4d1c677d47f3c48a4323f506927383d3,19TH_3,900209,19TH_3,A2 20th and Broadway Entrance / Exit,A2 20th and Broadway Entrance / Exit,2,POINT(-122.2684978 37.80885915),POINT (-122.26850 37.80886)
358,Bay Area 511 BART Schedule,1ee91bdb321ed2f8e617449cdc1b770a,2026-01-14,4d1c677d47f3c48a4323f506927383d3,ASHB_2,904109,ASHB_2,Adeline Street East Entrance / Exit Elevator,Adeline Street East Entrance / Exit Elevator,2,POINT(-122.2695656 37.85275725),POINT (-122.26957 37.85276)
359,Bay Area 511 BART Schedule,1ee91bdb321ed2f8e617449cdc1b770a,2026-01-14,4d1c677d47f3c48a4323f506927383d3,CIVC_6,901409,CIVC_6,A2 Market Street & Hyde Street (SW) Entrance /...,A2 Market Street & Hyde Street (SW) Entrance /...,2,POINT(-122.4146697 37.77895821),POINT (-122.41467 37.77896)


In [51]:
# gdf.drop(columns=['date']).explore()

In [9]:
gdf.gtfs_dataset_name.unique()

array(['Bay Area 511 Sonoma-Marin Area Rail Transit Schedule',
       'San Diego Schedule', 'LA Metro Rail Schedule',
       'Bay Area 511 BART Schedule'], dtype=object)

In [10]:
gdf[gdf.gtfs_dataset_name == 'Bay Area 511 Muni Schedule'].parent_station.unique()

array([], dtype=object)

In [11]:
# gdf.drop(columns=['date']).explore(tiles='CartoDB Positron')

In [26]:
rail_ferry_brt = gcs_geopandas().read_parquet(f"{GCS_FILE_PATH}rail_brt_ferry.parquet")

In [27]:
rail_ferry_brt

,schedule_gtfs_dataset_key_primary,stop_id,stop_name,route_id,route_type,hqta_type,geometry
0,1165b1474df778cb0fc3ba9246e32035,ABE,Aberdeen Amtrak Station,88,2,major_stop_rail,POINT (3634080.474 1021286.801)
1,1165b1474df778cb0fc3ba9246e32035,ABE,Aberdeen Amtrak Station,42985,2,major_stop_rail,POINT (3634080.474 1021286.801)
2,1165b1474df778cb0fc3ba9246e32035,ABQ,Albuquerque Alvarado Transportation Center,51,2,major_stop_rail,POINT (1212593.823 -240549.356)
3,1165b1474df778cb0fc3ba9246e32035,ACD,Arcadia Valley Station,87,2,major_stop_rail,POINT (2548982.716 350922.484)
4,1165b1474df778cb0fc3ba9246e32035,ADM,Ardmore Amtrak Station,86,2,major_stop_rail,POINT (2088240.532 -173890.462)
...,...,...,...,...,...,...,...
2843,ff87c8304a909c02ef5045cc802864ab,186,Burbank Airport - North (AV Line),Antelope Valley Line,2,major_stop_rail,POINT (152028.756 -421948.288)
2844,ff87c8304a909c02ef5045cc802864ab,188,San Bernardino - Tippecanoe,San Bernardino Line,2,major_stop_rail,POINT (252826.578 -434360.739)
2845,ff87c8304a909c02ef5045cc802864ab,189,Redlands - Esri,San Bernardino Line,2,major_stop_rail,POINT (258770.096 -435669.273)
2846,ff87c8304a909c02ef5045cc802864ab,190,Redlands - Downtown,San Bernardino Line,2,major_stop_rail,POINT (259874.998 -435755.157)


In [17]:
analysis_date

'2026-01-14'

In [38]:
keep_cols = ['schedule_gtfs_dataset_key_primary', 'stop_id', 'parent_station', 'stop_name', 'geometry']

In [44]:
gdf = gdf[keep_cols]

In [46]:
gdf

,schedule_gtfs_dataset_key_primary,stop_id,parent_station,stop_name,geometry
0,7ada6c55e4a29f4535e84c504a994b14,80105A,80105S,Anaheim Street Station - Long Beach & 12th Ent...,POINT (-118.18939 33.78135)
1,7ada6c55e4a29f4535e84c504a994b14,80138A,80138S,17th Street / SMC Station - East Ramp,POINT (-118.47982 34.02363)
2,7ada6c55e4a29f4535e84c504a994b14,80414A,80414S,Highland Park Station - Ave 57 Entrance,POINT (-118.19356 34.11070)
3,7ada6c55e4a29f4535e84c504a994b14,80405A,80405S,Soto Station - Main Entrance,POINT (-118.21019 34.04364)
4,7ada6c55e4a29f4535e84c504a994b14,80702A,80702S,LAX / Metro Transit Center - Main Pedestrian E...,POINT (-118.37816 33.95002)
...,...,...,...,...,...
356,1ee91bdb321ed2f8e617449cdc1b770a,ORIN_1,903209,Main Entrance / Exit,POINT (-122.18408 37.87812)
357,1ee91bdb321ed2f8e617449cdc1b770a,19TH_3,900209,A2 20th and Broadway Entrance / Exit,POINT (-122.26850 37.80886)
358,1ee91bdb321ed2f8e617449cdc1b770a,ASHB_2,904109,Adeline Street East Entrance / Exit Elevator,POINT (-122.26957 37.85276)
359,1ee91bdb321ed2f8e617449cdc1b770a,CIVC_6,901409,A2 Market Street & Hyde Street (SW) Entrance /...,POINT (-122.41467 37.77896)


In [48]:
rail_ferry_brt.query("stop_name.str.contains('Powell')")

,schedule_gtfs_dataset_key_primary,stop_id,stop_name,route_id,route_type,hqta_type,geometry
1241,1451f537bdcefd0e8ba827d12c4ef4b8,14757,Geary St & Powell St,38,3,major_stop_brt,POINT (-211788.651 -22763.113)
1242,1451f537bdcefd0e8ba827d12c4ef4b8,14757,Geary St & Powell St,38R,3,major_stop_brt,POINT (-211788.651 -22763.113)
1262,1451f537bdcefd0e8ba827d12c4ef4b8,15174,Jefferson St & Powell St,F,0,major_stop_rail,POINT (-212165.291 -20396.812)
1263,1451f537bdcefd0e8ba827d12c4ef4b8,15174,Jefferson St & Powell St,FBUS,0,major_stop_rail,POINT (-212165.291 -20396.812)
1301,1451f537bdcefd0e8ba827d12c4ef4b8,15417,Metro Powell Station/Downtown,K,0,major_stop_rail,POINT (-211681.672 -23071.415)
1302,1451f537bdcefd0e8ba827d12c4ef4b8,15417,Metro Powell Station/Downtown,L,0,major_stop_rail,POINT (-211681.672 -23071.415)
1303,1451f537bdcefd0e8ba827d12c4ef4b8,15417,Metro Powell Station/Downtown,J,0,major_stop_rail,POINT (-211681.672 -23071.415)
1304,1451f537bdcefd0e8ba827d12c4ef4b8,15417,Metro Powell Station/Downtown,M,0,major_stop_rail,POINT (-211681.672 -23071.415)
1305,1451f537bdcefd0e8ba827d12c4ef4b8,15417,Metro Powell Station/Downtown,N,0,major_stop_rail,POINT (-211681.672 -23071.415)
1468,1451f537bdcefd0e8ba827d12c4ef4b8,15817,O'Farrell St & Powell St,38R,3,major_stop_brt,POINT (-211769.630 -22883.584)


In [50]:
gdf[gdf.parent_station == '901302']

,schedule_gtfs_dataset_key_primary,stop_id,parent_station,stop_name,geometry
